In [ ]:
#fine tune model on med-easi dataset with flan-t5-base to get baseline model before doing RL

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)
import torch

#load Med-EASi dataset
ds = load_dataset("cbasu/Med-EASi")
print(ds)
print("Train sample:\n", ds["train"][0])



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/1397 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/196 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/300 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Expert', 'Simple', 'Annotation', 'sim', 'sentence_sim', 'compression', 'expert_fk_grade', 'expert_ari', 'layman_fk_grade', 'layman_ari', 'umls_expert', 'umls_layman', 'expert_terms', 'layman_terms', 'idx'],
        num_rows: 1397
    })
    validation: Dataset({
        features: ['Expert', 'Simple', 'Annotation', 'sim', 'sentence_sim', 'compression', 'expert_fk_grade', 'expert_ari', 'layman_fk_grade', 'layman_ari', 'umls_expert', 'umls_layman', 'expert_terms', 'layman_terms', 'idx'],
        num_rows: 196
    })
    test: Dataset({
        features: ['Expert', 'Simple', 'Annotation', 'sim', 'sentence_sim', 'compression', 'expert_fk_grade', 'expert_ari', 'layman_fk_grade', 'layman_ari', 'umls_expert', 'umls_layman', 'expert_terms', 'layman_terms', 'idx'],
        num_rows: 300
    })
})
Train sample:
 {'Expert': '75-90 % of the affected people have mild intellectual disability.', 'Simple': "People with syndromic intellectual disabi

Here, I retrain our flan-t5-base as our baseline before attempting to improve output with RL

In [3]:
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(batch):
    inputs = batch["Expert"]
    targets = batch["Simple"]

    model_inputs = tokenizer(
        inputs,
        max_length=256,
        truncation=True,
    )

    labels = tokenizer(
        text_target=targets,
        max_length=256,
        truncation=True,
    )["input_ids"]

    model_inputs["labels"] = labels
    return model_inputs

cols_to_remove = ds["train"].column_names

tokenized_ds = ds.map(
    preprocess,
    batched=True,
    remove_columns=cols_to_remove,
)

tokenized_ds

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/1397 [00:00<?, ? examples/s]

Map:   0%|          | 0/196 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1397
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 196
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 300
    })
})

In [4]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("Using device:", device)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Using device: cuda


In [5]:
import wandb
wandb.init(project="Med-EASE", name="flan-t5-base-rl-baseline")

#using best args from experiment
training_args = Seq2SeqTrainingArguments(
    output_dir="checkpoints/flan_t5_baseline",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=1,
    learning_rate=3e-5,
    num_train_epochs=5,
    warmup_steps=1000,
    logging_steps=50,
    save_steps=500,
    eval_steps=500,
    load_best_model_at_end=True,
    tpu_num_cores=8,
    predict_with_generate=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

train_result = trainer.train()

best_ckpt = trainer.state.best_model_checkpoint
best_metric = trainer.state.best_metric

print("Best checkpoint path:", best_ckpt)
print("Best metric", best_metric)

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sriyamed1 (sriyamed1-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/tmp/ipython-input-2813210795.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss
1,1.593800,1.883344
2,1.559400,1.832589
3,1.470800,1.795272
4,1.338600,1.776446
5,1.287000,1.767708


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


Best checkpoint path: checkpoints/flan_t5_baseline/checkpoint-875
Best metric 1.7677078247070312


In [6]:
!ls checkpoints/flan_t5_baseline

checkpoint-175	checkpoint-525	checkpoint-875
checkpoint-350	checkpoint-700	runs


In [7]:
wandb.finish()

eval/loss,█▅▃▂▁
eval/runtime,█▁▄▃▄
eval/samples_per_second,▁█▅▆▅
eval/steps_per_second,▁█▅▆▅
train/epoch,▁▁▂▂▂▃▃▄▄▄▄▅▅▅▆▆▇▇▇▇███
train/global_step,▁▁▂▂▂▃▃▄▄▄▄▅▅▅▆▆▇▇▇▇███
train/grad_norm,█▅▆▄▄▂▆▂▃▃▃▄▁▄▄▁▃
train/learning_rate,▁▁▂▂▃▃▄▄▄▅▅▆▆▇▇██
train/loss,█▇▆▅▅▅▅▃▄▄▃▃▄▂▂▂▁
eval/loss,1.76771
eval/runtime,3.6608


In [8]:
import numpy as np

test_ds = ds["test"]

def generate_baseline(batch):
    inputs = tokenizer(
        batch["Expert"],
        max_length=256,
        truncation=True,
        padding=True,
        return_tensors="pt",
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    model.eval()
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=180,
            do_sample=False,
        )

    batch["pred_baseline"] = tokenizer.batch_decode(
        outputs, skip_special_tokens=True
    )
    return batch

predictions_baseline = test_ds.map(
    generate_baseline,
    batched=True,
    batch_size=8,
)


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [9]:
!pip install evaluate
!pip install sacremoses sacrebleu
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=739680695d386e35d03ffd8bc638c20754c1df35c880dd1b6fb6425e7c35e5a1
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [10]:
import evaluate

sari_metric = evaluate.load("sari")
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")

sari_base = sari_metric.compute(
    sources=predictions_baseline["Expert"],
    predictions=predictions_baseline["pred_baseline"],
    references=[[ref] for ref in predictions_baseline["Simple"]],
)

bleu_base = bleu_metric.compute(
    predictions=predictions_baseline["pred_baseline"],
    references=[[ref] for ref in predictions_baseline["Simple"]],
)

rouge_base = rouge_metric.compute(
    predictions=predictions_baseline["pred_baseline"],
    references=predictions_baseline["Simple"],
)

print("BASELINE METRICS")
print("SARI:", sari_base)
print("BLEU:", bleu_base)
print("ROUGE:", rouge_base)


BASELINE METRICS
SARI: {'sari': 47.23916039277024}
BLEU: {'bleu': 0.2964113224231506, 'precisions': [0.5337995337995338, 0.3483483483483483, 0.2852233676975945, 0.2480450422270879], 'brevity_penalty': 0.8752213210898333, 'length_ratio': 0.8823956442831216, 'translation_length': 7293, 'reference_length': 8265}
ROUGE: {'rouge1': np.float64(0.5044537619176339), 'rouge2': np.float64(0.34804808963175793), 'rougeL': np.float64(0.46862529276683085), 'rougeLsum': np.float64(0.46752514254063837)}


In [13]:
import pandas as pd

df = pd.DataFrame({
    "expert": predictions_baseline["Expert"],
    "reference_simple": predictions_baseline["Simple"],
    "model_simple": predictions_baseline["pred_baseline"]
})

df.to_csv("model_outputs.csv", index=False)

In [11]:
!pip install textstat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 52.8 MB/s eta 0:00:0000:01


This section of the notebook is now exploring reinforcement learning inspired ways to get output focused more on readability. 

In [ ]:
import math
import textstat
import ast

#favor outputs that have lower FKGL level 
def readability_reward(text: str, min_grade: float = 5.0, max_grade: float = 15.0) -> float:
    """
    Return score between 0 and 1, where 1 = easiest to read, 0 = very hard
    map Flesch-Kincaid grade from [min_grade, max_grade] to [1,0].
    """
    grade = textstat.flesch_kincaid_grade(text)
    if math.isnan(grade):
        return 0.0

    grade = max(min_grade, min(max_grade, grade))
    score = 1.0 - (grade - min_grade) / (max_grade - min_grade)
    return score


In [ ]:
def parse_expert_terms(s: str):
    """
    expert_terms is stored like "['intellectual disability']" within med-easi dataset.
    parse it into a list and convert to lowercase
    """
    if s is None:
        return []
    try:
        terms = ast.literal_eval(s)
        if isinstance(terms, list):
            return [t.lower() for t in terms]
    except Exception:
        pass
    return []


def entity_reward(prediction: str, expert_terms_str: str) -> float:
    """
    return fraction of expert terms present in the prediction output.
    """
    terms = parse_expert_terms(expert_terms_str)
    if not terms:
        return 1.0

    pred_lower = prediction.lower()
    present = sum(int(term in pred_lower) for term in terms)
    return present / len(terms)


In [ ]:
#favor outputs that are less of a direct copy of expert field
def noncopy_reward(prediction: str, expert: str) -> float:
    """
    reward b/w 0 and 1, where 1 = very different from Expert field,
    0 = identical / heavily overlapping from Expert field.
    """
    pred_tokens = set(prediction.lower().split())
    expert_tokens = set(expert.lower().split())
    if not pred_tokens or not expert_tokens:
        return 0.0
    jaccard = len(pred_tokens & expert_tokens) / len(pred_tokens | expert_tokens) #use jaccard similarity
    #high overlap ==> low reward
    return 1.0 - jaccard


In [15]:
#favor outputs shorter than expert field
def length_reward(prediction: str, expert: str) -> float:
    """
    Reward b/w 0 and 1: 1 if prediction is much shorter than expert,
    decreasing as it approaches or exceeds expert length.
    """
    len_pred = len(prediction.split())
    len_expert = len(expert.split())
    if len_expert == 0:
        return 0.0

    ratio = len_pred / len_expert  #want this < 1 ideally
    if ratio >= 1.0:
        return 0.0
    #if ratio <= 0.5 -> ~1.0, if ratio approaching 1.0 -> 0.0
    return 1.0 - (ratio - 0.5) / 0.5 if ratio > 0.5 else 1.0


In [ ]:

def combined_reward(prediction, expert, expert_terms_str, w_read=0.40, w_ent=0.15, w_noncopy=0.35, w_len=0.10):
    r_read = readability_reward(prediction)
    r_ent  = entity_reward(prediction, expert_terms_str)
    r_nc   = noncopy_reward(prediction, expert)
    r_len  = length_reward(prediction, expert)

    return (
        w_read * r_read +
        w_ent  * r_ent +
        w_noncopy * r_nc +
        w_len * r_len
    )




In [ ]:
def generate_with_reranking(batch, num_candidates = 4):
    experts = batch["Expert"]
    expert_terms_list = batch["expert_terms"]

    preds_rl = []

    model.eval()
    for expert, expert_terms_str in zip(experts, expert_terms_list):
        inputs = tokenizer(
            expert,
            return_tensors="pt",
            truncation=True,
            max_length=256,
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=256,
                do_sample=True,
                top_k=50,
                top_p=0.9,
                temperature=1.2,
                num_return_sequences=num_candidates,
            )

        candidates = tokenizer.batch_decode(outputs, skip_special_tokens=True)

        best_cand = None
        best_reward = -1.0
        for cand in candidates: #for each candidate, compute reward and select best candidate to use as the final prediction
            r = combined_reward(
                prediction=cand,
                expert=expert,
                expert_terms_str=expert_terms_str,
            )
            if r > best_reward:
                best_reward = r
                best_cand = cand

        preds_rl.append(best_cand)

    batch["pred_rl"] = preds_rl
    return batch

predictions_rl = ds["test"].map(
    generate_with_reranking,
    batched=True,
    batch_size=4,
)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [18]:
sari_rl = sari_metric.compute(
    sources=predictions_rl["Expert"],
    predictions=predictions_rl["pred_rl"],
    references=[[ref] for ref in predictions_rl["Simple"]],
)

bleu_rl = bleu_metric.compute(
    predictions=predictions_rl["pred_rl"],
    references=[[ref] for ref in predictions_rl["Simple"]],
)

rouge_rl = rouge_metric.compute(
    predictions=predictions_rl["pred_rl"],
    references=predictions_rl["Simple"],
)

print("RL-RERANKED METRICS")
print("SARI:", sari_rl)
print("BLEU:", bleu_rl)
print("ROUGE:", rouge_rl)


RL-RERANKED METRICS
SARI: {'sari': 36.83204616110464}
BLEU: {'bleu': 0.12480641895779039, 'precisions': [0.39967266775777416, 0.1855421686746988, 0.13012704174228676, 0.10307101727447217], 'brevity_penalty': 0.7027876530867181, 'length_ratio': 0.7392619479733817, 'translation_length': 6110, 'reference_length': 8265}
ROUGE: {'rouge1': np.float64(0.32887475439493924), 'rouge2': np.float64(0.16760471006347882), 'rougeL': np.float64(0.2878747139054525), 'rougeLsum': np.float64(0.2882481383411188)}


In [30]:
import pandas as pd

df = pd.DataFrame({
    "expert": predictions_rl["Expert"],
    "reference_simple": predictions_rl["Simple"],
    "model_simple": predictions_rl["pred_rl"]
})

df.to_csv("model_outputs_rl.csv", index=False)

In [19]:
for i in range(10):
    print(f"\n=== Example {i} ===")
    print("EXPERT:", predictions_rl["Expert"][i])
    print("SIMPLE (ref):", predictions_rl["Simple"][i])
    print("BASELINE:", predictions_baseline["pred_baseline"][i])
    print("RL-RERANK:", predictions_rl["pred_rl"][i])

    print("Baseline readability:", readability_reward(predictions_baseline["pred_baseline"][i]))
    print("RL readability:", readability_reward(predictions_rl["pred_rl"][i]))

    print("Baseline entity:", entity_reward(predictions_baseline["pred_baseline"][i], predictions_rl["expert_terms"][i]))
    print("RL entity:", entity_reward(predictions_rl["pred_rl"][i], predictions_rl["expert_terms"][i]))



=== Example 0 ===
EXPERT: Intervention for obese adolescents should be focused on developing healthy eating and exercise habits rather than on losing a specific amount of weight.
SIMPLE (ref): The treatment of adolescent obesity is focused on developing healthy eating and exercise habits rather than on losing a specific amount of weight.
BASELINE: Intervention for obese adolescents should be focused on developing healthy eating and exercise habits rather than on losing a specific amount of weight.
RL-RERANK: Weight loss for obese adolescents should not be about losing a particular amount of weight.
Baseline readability: 0.0
RL readability: 0.5073333333333331
Baseline entity: 0.8571428571428571
RL entity: 0.42857142857142855

=== Example 1 ===
EXPERT: The liver may be enlarged, hard, or tender; massive hepatomegaly with easily palpable nodules signifies advanced disease.
SIMPLE (ref): Typically, the liver is enlarged and hard. It may feel tender and often lumpy.
BASELINE: The liver may